In [4]:
import numpy as np
import pandas as pd
import itertools
from Bio import SeqIO #pip install biopython

In [5]:
negative_set = pd.read_csv('negative_set.bed', delimiter='\t', header = None)
gm = pd.read_csv('GM12878.bed', delimiter='\t', header=None)

In [3]:
negative_set.head()

,0,1,2
0,chr10,32251,36771
1,chr10,39431,39891
2,chr10,72312,74222
3,chr10,84717,85177
4,chr10,90499,91949


In [4]:
gm.head()

,0,1,2,3
0,chr1,773300,774100,7.866088
1,chr1,778980,779450,6.472419
2,chr1,800100,802000,11.010675
3,chr1,825670,826410,6.114487
4,chr1,839470,842590,8.848865


In [5]:
train_neg = negative_set.loc[~negative_set[0].isin(['chrY', 'chrX'])]

In [6]:
GRCh37 = pd.read_csv('GRCh37.primary_assembly.genome.fa', sep='\t')

PermissionError: [Errno 13] Permission denied: 'GRCh37.primary_assembly.genome.fa'

In [8]:
GRCh37_1 = GRCh37.loc[~GRCh37['>chr1 1'].str.contains('NN')]

In [10]:
GRCh37_1

,>chr1 1
167,ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC...
168,ACCCTAACCCTAACCCTAACCCTAACCCAACCCTAACCCTAACCCT...
169,CCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCTAACCCT...
170,CCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCC...
171,TAACCCTAACCCTAACCCTAACCCTAACCCCAACCCCAACCCCAAC...
...,...
51696864,GTTGTAGTGCTGGCTGTTGTTGTGGTTGTGGGTTCAGTAGAGGTGG...
51696865,GTGGTTGTGGTCTCAGTAGCAGTGGCTGCAGTTGTGGTGGCAGACT...
51696866,CTTGCTGTGGTTGTCATTGTGGGTTCAGCAGAAGTGGCAGTGGTGG...
51696867,GTCTCAGTGGAAGTGCCTGCAGTTGTGGTGGCAGACTGTTCAGTTG...


In [10]:
def Find_Kmers(string, k):    
    kmers = itertools.product('ACTG', repeat=k)
    for i in kmers:
        if string.count(''.join(i)) > 0:
            print(f"{''.join(i)}: {string.count(''.join(i))}")


In [15]:
itertools.product('ACTG', repeat=3)

In [16]:
Find_Kmers('ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC', 3)

AAC: 7
ACC: 8
CCC: 8
CCT: 7
CTA: 7
TAA: 7


In [9]:
def Kmers_cols(column, k):
    kmers = itertools.product('ACTG', repeat=k)
    df = pd.DataFrame()
    for i in kmers:
        kmer = ''.join(i)
        rev_kmer = kmer.replace('A', '%temp%').replace('T', 'A').replace('%temp%', 'T').replace('C', '%temp%').replace('G', 'C').replace('%temp%', 'G')
        if(rev_kmer not in df.columns):
            df[kmer] = column.str.count(kmer)/(len(column.iloc[0])-k+1)
        else:
            df[rev_kmer] += column.str.count(kmer)/(len(column.iloc[0])-k+1)
    return df

In [ ]:
df_3mers = Kmers_cols(GRCh37_1.head(100)['>chr1 1'], 4)
df_3mers.head()

In [90]:
Kmers_cols(GRCh37_1.head(100)['>chr1 1'], 4)

/var/folders/dc/tm97_rl10kj62c4gsppshgm80000gn/T/ipykernel_8546/3841867262.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[kmer] = column.str.count(kmer)/(len(column.iloc[0])-k+1)
/var/folders/dc/tm97_rl10kj62c4gsppshgm80000gn/T/ipykernel_8546/3841867262.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[kmer] = column.str.count(kmer)/(len(column.iloc[0])-k+1)
/var/folders/dc/tm97_rl10kj62c4gsppshgm80000gn/T/ipykernel_8546/3841867262.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

,AAAA,AAAC,AAAT,AAAG,AACA,AACC,AACT,AACG,AATA,AATC,...,CGCT,CGCG,CGTA,CGTC,CGTT,CGTG,CGGA,CGGC,CGGT,CGGG
167,0.0,0.000000,0.0,0.000000,0.000000,0.157895,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
168,0.0,0.000000,0.0,0.000000,0.000000,0.157895,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
169,0.0,0.000000,0.0,0.000000,0.000000,0.157895,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
170,0.0,0.035088,0.0,0.000000,0.000000,0.157895,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
171,0.0,0.000000,0.0,0.000000,0.000000,0.175439,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.017544,0.0,0.0,...,0.017544,0.000000,0.0,0.000000,0.017544,0.0,0.000000,0.000000,0.000000,0.035088
263,0.0,0.000000,0.0,0.017544,0.000000,0.000000,0.0,0.017544,0.0,0.0,...,0.035088,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.017544
264,0.0,0.000000,0.0,0.000000,0.000000,0.035088,0.0,0.017544,0.0,0.0,...,0.000000,0.017544,0.0,0.035088,0.000000,0.0,0.035088,0.017544,0.000000,0.017544
265,0.0,0.000000,0.0,0.000000,0.000000,0.017544,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.017544,0.017544,0.035088,0.017544


Moim zdaniem, wczytanie GRCh37:

In [6]:
records = list(SeqIO.parse('GRCh37_primary_assembly_genome.fa', "fasta"))
indicators = ['Seq']
table = pd.DataFrame(columns = indicators)
for rec in records:
    table.loc[rec.name] = [rec.seq]
table = table.reset_index()    
good_chr = ['chr' + str(i) for i in range(1,22)]
df = table.loc[table['index'].isin(good_chr),:]
df['Seq_with_no_N'] = [seq.replace('N', '') for seq in df['Seq'] ]

<ipython-input-6-2a089a30b53d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Seq_with_no_N'] = [seq.replace('N', '') for seq in df['Seq'] ]


In [7]:
df

,index,Seq,Seq_with_no_N
0,chr1,"(N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...","(T, A, A, C, C, C, T, A, A, C, C, C, T, A, A, ..."
1,chr10,"(N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...","(G, A, A, T, T, C, C, T, T, G, A, G, G, C, C, ..."
2,chr11,"(N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...","(G, A, A, T, T, C, T, A, C, A, T, T, A, G, A, ..."
3,chr12,"(N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...","(A, G, G, C, T, G, G, C, C, T, G, C, T, C, T, ..."
4,chr13,"(N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...","(G, A, A, T, T, C, A, A, C, T, G, C, C, T, C, ..."
5,chr14,"(N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...","(G, A, A, T, T, C, T, T, G, T, G, T, G, T, A, ..."
6,chr15,"(N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...","(T, A, A, T, T, T, T, T, G, T, G, T, G, A, A, ..."
7,chr16,"(N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...","(T, A, A, C, C, C, T, A, A, C, C, C, T, A, A, ..."
8,chr17,"(A, A, G, C, T, T, C, T, C, A, C, C, C, T, G, ...","(A, A, G, C, T, T, C, T, C, A, C, C, C, T, G, ..."
9,chr18,"(N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...","(C, C, C, T, A, A, C, C, C, T, A, A, C, C, C, ..."


In [11]:
Kmers_cols(df['Seq_with_no_N'], 4)

,AAAA,AAAC,AAAT,AAAG,AACA,AACC,AACT,AACG,AATA,AATC,...,CGCT,CGCG,CGTA,CGTC,CGTT,CGTG,CGGA,CGGC,CGGT,CGGG
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
